# Lab 6: Multi-Agent Orchestration



## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [2]:
from helper import nb_print

In [3]:
from letta import create_client 

client = create_client()

Saved Config:  /home/jovyan/.letta/config
📖 Letta configuration file updated!
🧠 model	-> gpt-4
🖥️  endpoint	-> http://jupyter-api-proxy.internal.dlai/rev-proxy/letta
Saved Config:  /home/jovyan/.letta/config
Saved Config:  /home/jovyan/.letta/config


In [4]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 

## Section 1: Shared Memory Block

In [5]:
from letta.schemas.block import Block 

org_description= "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

org_block = Block(name="company", value=org_description )

In [6]:
from letta.schemas.memory import BasicBlockMemory

class OrgMemory(BasicBlockMemory): 

    def __init__(self, persona: str, org_block: Block): 
        persona_block = Block(name="persona", value=persona)
        super().__init__(blocks=[persona_block, org_block])

In [7]:
#cleanup. This code will remove agents if the code is run more than once otherwise it will do nothing.
for agent in client.list_agents(): 
    client.delete_agent(agent.id)

In [8]:
# client.delete_agent(recruiter_agent.id)  #cleanup code for repeated runs if needed

## Section 2: Orchestrating Multiple Agents 

#### Evaluator Agent

In [9]:
def read_resume(self, name: str): 
    """
    Read the resume data for a candidate given the name

    Args: 
        name (str): Candidate name 

    Returns: 
        resume_data (str): Candidate's resume data 
    """
    import os
    filepath = os.path.join("data", "resumes", name.lower().replace(" ", "_") + ".txt")
    #print("read", filepath)
    return open(filepath).read()

In [10]:
def submit_evaluation(
    self, 
    candidate_name: str, 
    reach_out: bool, 
    resume: str, 
    justification: str
): 
    """
    Submit a candidate for outreach. 

    Args: 
        candidate_name (str): The name of the candidate
        reach_out (bool): Whether to reach out to the candidate
        resume (str): The text representation of the candidate's resume 
        justification (str): Justification for reaching out or not
    """
    from letta import create_client 
    client = create_client()

    message = "Reach out to the following candidate. " \
    + f"Name: {candidate_name}\n" \
    + f"Resume Data: {resume}\n" \
    + f"Justification: {justification}"
    print("eval agent", candidate_name)
    if reach_out:
        response = client.send_message(
            agent_name="outreach_agent", 
            role="user", 
            message=message
        ) 
    else: 
        print(f"Candidate {candidate_name} is rejected: {justification}")

In [11]:
read_resume_tool = client.create_tool(read_resume) 
submit_evaluation_tool = client.create_tool(submit_evaluation)

In [12]:
skills = "Front-end (React, Typescript), software engineering " \
+ "(ideally Python), and experience with LLMs."
eval_persona = f"You are responsible to finding good recruiting " \
+ "candidates, for the company description. " \
+ f"Ideal canddiates have skills: {skills}. " \
+ "Submit your candidate evaluation with the submit_evaluation tool. "


eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tools=[read_resume_tool.name, submit_evaluation_tool.name]
)

#### Outreach agent 

In [13]:
def email_candidate(self, content: str): 
    """
    Send an email

    Args: 
        content (str): Content of the email 
    """
    print("Pretend to email:", content)
    return

email_candidate_tool = client.create_tool(email_candidate)

In [14]:
outreach_persona = "You are responsible for sending outbound emails " \
+ "on behalf of a company with the send_emails tool to " \
+ "potential candidates. " \
+ "If possible, make sure to personalize the email by appealing " \
+ "to the recipient with details about the company. " \
+ "You position is `Head Recruiter`, and you go by the name Bob, with contact info bob@gmail.com. " \
+ """
Follow this email template: 

Hi <candidate name>, 

<content> 

Best, 
<your name> 
<company name> 
"""

outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tools=[email_candidate_tool.name]
)

In [15]:
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message="Candidate: Tony Stark"
)

eval agent Tony Stark
Pretend to email: Hi Tony Stark, 

I'm Bob, the Head Recruiter at AgentOS. We're transforming the landscape of LLM agents - tools that can support a wide range of real-world tasks. Having come across your impressive professional profile, we wanted to learn more about you and your passion for web development. 

Your extensive experience with React, team leadership, and MIT background align with what we value at AgentOS. Although our work heavily involves Python and LLMs, we also work with React and believe your vast knowledge could contribute significantly to our team. In addition to that, learning and growth is part of our culture, and our team would provide complete support in bridging any gap in skills. 

We strongly believe in innovation and collaboration, and we think your profile matches this spirit. We hope you are open to exploring this opportunity further. Please feel free to reply to this email if you're interested and have any questions. Looking forward 

In [16]:
nb_print(response.messages)

In [17]:
feedback = "Our company pivoted to foundation model training"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

In [18]:
feedback = "The company is also renamed to FoundationAI"
response = client.send_message(
    agent_name="eval_agent", 
    role="user", 
    message=feedback
)

In [19]:
nb_print(response.messages)

In [20]:
response = client.send_message(
    agent_name="eval_agent", 
    role="system", 
    message="Candidate: Spongebob Squarepants"
)

eval agent Spongebob Squarepants
Pretend to email: Hi Spongebob Squarepants, 

My name is Bob and I'm the Head Recruiter at FoundationAI, a company at the forefront of foundation model training. Your rigorous academic profile and extensive contributions to AI research, particularly in agent technologies, is the kind of experience we highly value and seek at FoundationAI. 

We are deeply fascinated by your work, including the development of advanced agent-based models and your scholarly contributions to AI journals and conferences. We believe your expertise in Python programming and machine learning frameworks would be an extremely valuable asset to the projects we handle. 

At FoundationAI, we are committed to providing opportunities for professional growth and working on cutting-edge AI technologies. We would love to hear more about your passion for agent technology and discuss how you could lend your talents to our team. 

Please feel free to reply to this email if you're interested 

In [21]:
client.get_core_memory(outreach_agent.id).get_block("company")

Block(value='The company is now called FoundationAI, specializing in foundation model training.', limit=2000, name='company', template=False, label='company', description=None, metadata_={}, user_id=None, id='block-e0e05fb6-d785-4bdf-9358-795024d86833')

## Section 3: Adding an orchestrator agent 

In [22]:
#re-create agents 
client.delete_agent(eval_agent.id)
client.delete_agent(outreach_agent.id)

eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tools=[read_resume_tool.name, submit_evaluation_tool.name]
)

outreach_agent = client.create_agent(
    name="outreach_agent", 
    memory=OrgMemory(
        persona=outreach_persona, 
        org_block=org_block
    ), 
    tools=[email_candidate_tool.name]
)

In [23]:
client.get_block(org_block.id)

Block(value='The company is now called FoundationAI, specializing in foundation model training.', limit=2000, name='company', template=False, label='company', description=None, metadata_={}, user_id='user-2efb600b-8325-4546-af5d-6f1cbbbc3ad2', id='block-e0e05fb6-d785-4bdf-9358-795024d86833')

In [24]:
from typing import Optional

def search_candidates_db(self, page: int) -> Optional[str]: 
    """
    Returns 1 candidates per page. 
    Must start at page 0.
    Page 0 returns the first 1 candidate, 
    Page 1 returns the next 1, etc.
    Returns `None` if no candidates remain. 

    Args: 
        page (int): The page number to return candidates from 

    Returns: 
        candidate_names (List[str]): Names of the candidates
    """
    
    names = ["Tony Stark", "Spongebob Squarepants", "Gautam Fang"]
    if page >= len(names): 
        return None
    return names[page]

def consider_candidate(self, name: str): 
    """
    Submit a candidate for consideration. 

    Args: 
        name (str): Candidate name to consider 
    """
    from letta import create_client 
    client = create_client()
    message = f"Consider candidate {name}" 
    print("Sending message to eval agent: ", message)
    response = client.send_message(
        agent_name="eval_agent", 
        role="user", 
        message=message
    ) 


In [25]:
search_candidate_tool = client.create_tool(search_candidates_db)
consider_candidate_tool = client.create_tool(consider_candidate)

# create recruiter agent
recruiter_agent = client.create_agent(
    name="recruiter_agent", 
    memory=OrgMemory(
        persona="You run a recruiting process for a company. " \
        + "Your job is to continue to pull candidates from the " 
        + "`search_candidates_db` tool until there are no more " \
        + "candidates left. " \
        + "For each candidate, consider the candidate by calling "
        + "the `consider_candidate` tool. " \
        + "You should continue to call `search_candidates_db` " \
        + "followed by `consider_candidate` until there are no more " \
        " candidates. Start at page 0. ",
        org_block=org_block
    ), 
    tools=[search_candidate_tool.name, consider_candidate_tool.name]
)
   

In [26]:
response = client.send_message(
    agent_id=recruiter_agent.id, 
    role="system", 
    message="Run generation"
)

Sending message to eval agent:  Consider candidate Tony Stark
eval agent Tony Stark
Pretend to email: Hi Tony Stark,

I hope this message finds you well. I came across your profile and was impressed with your extensive experience as a Frontend Engineer, especially with React.

We are FoundationAI, a company specializing in foundation model training. We are currently looking for someone like yourself, who has a depth of understanding in web application development.

I'd love to have a chat about how you could bring your React expertise to our team. We believe your experience and technical skills could significantly elevate our projects. Also, your contribution at the Massachusetts Institute of Technology is quite commendable.

Let me know if you'd be open to a discussion about the opportunities here at FoundationAI. You can reach out to me at bob@gmail.com.


Best,
Bob
Head Recruiter
FoundationAI
Sending message to eval agent:  Consider candidate Spongebob Squarepants
eval agent Spongeb

In [27]:
nb_print(response.messages)